<a href="https://colab.research.google.com/github/parvathysarat/kg-qa/blob/master/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
class QuestionAnswering(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, entity_weights,rel_weights):
        super(QuestionAnswering, self).__init__()

        # entity embedding
        self.entity_embeddings = nn.Embedding.from_pretrained(self.entity_weights)  
        self.entity_embeddings.weight.requires_grad = False  
        # word embedding dimension = 50
        self.entity_linear = nn.Linear(in_features = 50,out_features = 50)

        # relation embedding
        self.relation_embeddings = nn.Embedding.from_pretrained(self.rel_weights)  
        self.relation_embeddings.weight.requires_grad = False          
        # entity dimension = 50
        word_dim, entity_dim = 50
        self.num_vocab = 13545
        self.relation_linear = nn.Linear(in_features = 2*word_dim, out_features = entity_dim)
        self.k=3
        for i in range(3):
            self.add_module('q2e_linear' + str(i), nn.Linear(in_features=entity_dim, out_features=entity_dim))
            self.add_module('d2e_linear' + str(i), nn.Linear(in_features=entity_dim, out_features=entity_dim))
            self.add_module('e2q_linear' + str(i), nn.Linear(in_features=self.k * entity_dim, out_features=entity_dim))
            self.add_module('e2d_linear' + str(i), nn.Linear(in_features=self.k * entity_dim, out_features=entity_dim))
            self.add_module('e2e_linear' + str(i), nn.Linear(in_features=self.k * entity_dim, out_features=entity_dim))
            
        
            self.add_module('kb_head_linear' + str(i), nn.Linear(in_features=entity_dim, out_features=entity_dim))
            self.add_module('kb_tail_linear' + str(i), nn.Linear(in_features=entity_dim, out_features=entity_dim))
            self.add_module('kb_self_linear' + str(i), nn.Linear(in_features=entity_dim, out_features=entity_dim))

        # initialize document embeddings
        
        self.doc_embedding = nn.Embedding(num_embeddings = num_vocab+1, embedding_dim = word_dim, padding_idx = sef.num_vocab)  
        self.lstm = nn.LSTM(embedding_dim, hidden_dim,dropout = DROPOUT, num_layers = LSTM_LAYERS, bidirection=True)

        # create LSTMs
        self.node_encoder = nn.LSTM(input_size = word_dim, hidden_size = entity_dim, batch_first = True)
        # self.q_rel_encoder = nn.LSTM(input_size = word_dim, hidden_size = entity_dim, batch_first = True)
        self.bi_doc_encoder = nn.LSTM(input_size = word_dim, hidden_size = entity_dim, batch_first = True, bidirectional=True)
        self.doc_info_carrier = nn.LSTM(input_size = entity_dim, hidden_size = entity_dim, batch_first = True,bidirectional=True)

        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        self.elu = nn.ELU()

        # dropout
        self.lstm_drop = nn.Dropout(p=lstm_dropout)
        self.linear_drop = nn.Dropout(p=linear_dropout)

        self.score_func = nn.Linear(in_features=entity_dim, out_features=1)
        
        self.log_softmax = nn.LogSoftmax(dim=1)
        self.softmax_d1 = nn.Softmax(dim=1)
        self.kld_loss = nn.KLDivLoss()
        self.bce_loss = nn.BCELoss()
        self.bce_loss_logits = nn.BCEWithLogitsLoss()



    def forward(self, batch):
        tag_scores = None
        # Implement the forward pass.
